In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from models.grapheme_to_phoneme import GraphemeToPhoneme
import tensorflow as tf
import numpy as np

# Define parametes

In [3]:
train_parameters = {
    "lr": 0.001,
    "decay_steps": 1000,
    "decay_rate": 0.85,
    "dropout_prob": 0.95
}

model_parameters = {
  "embedding_size": 80,
  "num_units": 10,
  "num_layers": 2,
  "num_beams": 1
}

input_vocab_size = 10
output_vocab_size= 11

end_token = 10

num_steps = 2

save_energy = 2

# Train model

In [4]:
with tf.Session() as sess:
    characters = tf.placeholder(tf.int32, [None, 200])
    characters_seq_len = tf.placeholder(tf.int32, [None])
    phonemes = tf.placeholder(tf.int32, [None, 200])
    phonemes_seq_len = tf.placeholder(tf.int32, [None])
    
    prediction_characters = tf.placeholder(tf.int32, [None, 200])
    prediction_characters_seq_len = tf.placeholder(tf.int32, [None])
    
    model = GraphemeToPhoneme(input_vocab_size, output_vocab_size, end_token, model_parameters)
    
    train_op_tf, loss_tf, global_step_tf, summary_tf, decoder_output_tf = model.build_train_operations(
        characters, characters_seq_len, phonemes, phonemes_seq_len, train_parameters
    )
    
    prediction_tf = model.build_prediction(prediction_characters, prediction_characters_seq_len, True)

    train_writer = tf.summary.FileWriter('../log/train_grapheme_to_phoneme_model_notebook/train', sess.graph)
    
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    saver = tf.train.Saver(max_to_keep=3, keep_checkpoint_every_n_hours=3)
    
    for _ in xrange(num_steps):
        out = sess.run([
            train_op_tf,
            global_step_tf,
            loss_tf,
            summary_tf,
            decoder_output_tf,
        ], feed_dict={
            characters: np.random.rand(2, 200),
            characters_seq_len: 10* np.ones((2)),
            phonemes: np.random.rand(2, 200),
            phonemes_seq_len: 8* np.ones((2))
        })
        _, global_step, loss, summary, _ = out

        train_writer.add_summary(summary, global_step)

        # detect gradient explosion
        if loss > 1e8 and global_step > 500:
            print('loss exploded')
            break

        if global_step % save_energy == 0 and global_step != 0:

            print('saving weights')
            if not os.path.exists('../weights/train_grapheme_to_phoneme_model_notebook/'):
                os.makedirs('../weights/train_grapheme_to_phoneme_model_notebook/')
            saver.save(sess, '../weights/train_grapheme_to_phoneme_model_notebook/', global_step=global_step)
    
    coord.request_stop()
    coord.join(threads)

saving weights
